# Lab Assignment 4: Exploration of Different Diff Algorithms on Open-Source Repositories

**Course:** CS202 Software Tools and Techniques for CSE  
**Date:** 25th August 2025

## Objective
The purpose of this lab is to explore differences in diff outputs for Open-Source Repositories in the wild. We analyze the impact of different diff algorithms (Myers vs Histogram) on code and non-code artifacts.

## Repository Selection
We selected three medium-to-large scale open-source repositories:
1. **Retrofit** - A type-safe HTTP client for Android and Java
2. **Glide** - An image loading and caching library for Android
3. **Timber** - A logger with a small, extensible API

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import re
from pydriller import Repository
from collections import Counter
import seaborn as sns

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

## Repository Setup

First, let's clone the selected repositories for analysis.

In [ ]:
# Clone the selected repositories
repositories = {
    'retrofit': 'https://github.com/square/retrofit.git',
    'glide': 'https://github.com/bumptech/glide.git',
    'timber': 'https://github.com/JakeWharton/timber.git'
}

# Clone repositories (uncomment if not already cloned)
# for name, url in repositories.items():
#     !git clone {url} {name}
#     print(f"Cloned {name} repository")

print("Repository setup complete")

## Diff Algorithm Implementation

We implement functions to extract diffs using both Myers (default) and Histogram algorithms, then compare their outputs.

In [ ]:
def get_myers_diff(repo_path, commit_hash, parent_hash, file_path):
    """Get diff using Myers algorithm (git default)."""
    try:
        result = subprocess.run(
            ["git", "-C", repo_path, "diff", parent_hash, commit_hash, "--", file_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True,
            errors="replace"
        )
        
        # Extract only the actual diff content (skip headers)
        lines = result.stdout.splitlines()
        diff_content = []
        
        include_line = False
        for line in lines:
            if line.startswith("@@"):
                include_line = True
            if include_line:
                diff_content.append(line)
        
        return "\n".join(diff_content).strip()
    
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr.strip()}"
    except Exception as e:
        return f"Error: {str(e)}"

def get_histogram_diff(repo_path, commit_hash, parent_hash, file_path):
    """Get diff using Histogram algorithm."""
    try:
        result = subprocess.run(
            ["git", "-C", repo_path, "diff", "--histogram", parent_hash, commit_hash, "--", file_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True,
            errors="replace"
        )
        
        # Extract only the actual diff content (skip headers)
        lines = result.stdout.splitlines()
        diff_content = []
        
        include_line = False
        for line in lines:
            if line.startswith("@@"):
                include_line = True
            if include_line:
                diff_content.append(line)
        
        return "\n".join(diff_content).strip()
    
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr.strip()}"
    except Exception as e:
        return f"Error: {str(e)}"

## File Classification

We classify files into different categories to analyze the impact of diff algorithms on different types of artifacts.

In [ ]:
def classify_file_type(file_path):
    """Classify files into different categories based on path and extension."""
    if not file_path:
        return "unknown"
    
    path_lower = file_path.lower()
    
    # Test files
    if any(test_indicator in path_lower for test_indicator in ['test', 'spec', '/tests/', '/test/']):
        return "test"
    
    # Source code files
    code_extensions = ['.java', '.kt', '.py', '.js', '.ts', '.cpp', '.c', '.h', '.cs']
    if any(path_lower.endswith(ext) for ext in code_extensions):
        return "source_code"
    
    # Build and configuration files
    build_files = ['build.gradle', 'pom.xml', 'package.json', 'makefile', 'cmake']
    config_extensions = ['.gradle', '.xml', '.json', '.yaml', '.yml', '.properties', '.config']
    if any(build_file in path_lower for build_file in build_files) or \
       any(path_lower.endswith(ext) for ext in config_extensions):
        return "build_config"
    
    # Documentation files
    doc_files = ['readme', 'changelog', 'license', 'contributing']
    doc_extensions = ['.md', '.txt', '.rst', '.adoc']
    if any(doc_file in path_lower for doc_file in doc_files) or \
       any(path_lower.endswith(ext) for ext in doc_extensions):
        return "documentation"
    
    # Resource files
    resource_extensions = ['.png', '.jpg', '.jpeg', '.gif', '.svg', '.ico', '.css', '.html']
    if any(path_lower.endswith(ext) for ext in resource_extensions):
        return "resources"
    
    return "other"

# Test the classification function
test_files = [
    "src/main/java/com/example/Main.java",
    "src/test/java/com/example/MainTest.java",
    "build.gradle",
    "README.md",
    "app/src/main/res/drawable/icon.png"
]

print("File classification examples:")
for file_path in test_files:
    print(f"{file_path} -> {classify_file_type(file_path)}")

## Repository Analysis Function

Main function to analyze a repository and compare diff algorithms.

In [ ]:
def analyze_repository_diffs(repo_path, repo_name, commit_limit=200):
    """Analyze diff discrepancies in a repository."""
    
    print(f"\n=== Analyzing {repo_name} repository ===")
    
    results = []
    commits_processed = 0
    total_files = 0
    
    try:
        for commit in Repository(repo_path, histogram_diff=False).traverse_commits():
            if commits_processed >= commit_limit:
                break
            
            # Skip commits without parents (initial commit)
            if not commit.parents:
                continue
            
            parent_sha = commit.parents[0]
            
            for modified_file in commit.modified_files:
                file_path = modified_file.new_path or modified_file.old_path
                
                if not file_path:
                    continue
                
                # Get diffs using both algorithms
                histogram_diff = get_histogram_diff(repo_path, commit.hash, parent_sha, file_path)
                myers_diff = get_myers_diff(repo_path, commit.hash, parent_sha, file_path)
                
                # Skip empty diffs
                if not histogram_diff.strip() or not myers_diff.strip():
                    continue
                
                # Check for discrepancy
                has_discrepancy = histogram_diff != myers_diff
                file_category = classify_file_type(file_path)
                
                results.append({
                    "repository": repo_name,
                    "commit_hash": commit.hash,
                    "parent_hash": parent_sha,
                    "file_path": file_path,
                    "file_category": file_category,
                    "commit_message": commit.msg,
                    "histogram_diff": histogram_diff,
                    "myers_diff": myers_diff,
                    "has_discrepancy": has_discrepancy,
                    "histogram_diff_size": len(histogram_diff),
                    "myers_diff_size": len(myers_diff)
                })
                
                total_files += 1
            
            commits_processed += 1
            
            if commits_processed % 50 == 0:
                print(f"Processed {commits_processed} commits, {total_files} files")
    
    except Exception as e:
        print(f"Error analyzing repository {repo_name}: {e}")
    
    print(f"Analysis complete: {commits_processed} commits, {total_files} files")
    return results

## Running Analysis on Selected Repositories

In [ ]:
# Analyze all three repositories
all_results = []
repository_paths = ['retrofit', 'glide', 'timber']

for repo_name in repository_paths:
    try:
        repo_results = analyze_repository_diffs(repo_name, repo_name, commit_limit=100)
        all_results.extend(repo_results)
        
        # Save individual repository results
        if repo_results:
            repo_df = pd.DataFrame(repo_results)
            repo_df.to_csv(f"{repo_name}_diff_analysis.csv", index=False)
            print(f"Saved {len(repo_results)} results for {repo_name}")
    
    except Exception as e:
        print(f"Failed to analyze {repo_name}: {e}")

# Create master dataframe
if all_results:
    master_df = pd.DataFrame(all_results)
    master_df.to_csv("diff_algorithms_comparison.csv", index=False)
    print(f"\nMaster dataset created with {len(all_results)} total entries")
else:
    print("No results collected")

## Analysis and Visualization

Analyzing the discrepancies between diff algorithms and visualizing the results.

In [ ]:
# Load results for analysis
if 'master_df' in locals() and not master_df.empty:
    df = master_df
else:
    # Try to load from CSV if available
    try:
        df = pd.read_csv("diff_algorithms_comparison.csv")
    except FileNotFoundError:
        print("No data available for analysis")
        df = pd.DataFrame()

if not df.empty:
    print("=== Diff Algorithm Comparison Analysis ===")
    print(f"Total files analyzed: {len(df):,}")
    print(f"Files with discrepancies: {df['has_discrepancy'].sum():,}")
    print(f"Discrepancy rate: {(df['has_discrepancy'].sum() / len(df)) * 100:.2f}%")
    
    # Repository-wise analysis
    print("\n=== Repository-wise Analysis ===")
    repo_analysis = df.groupby('repository').agg({
        'has_discrepancy': ['count', 'sum'],
        'file_category': 'count'
    }).round(2)
    
    repo_analysis.columns = ['total_files', 'discrepancies', 'files_analyzed']
    repo_analysis['discrepancy_rate'] = (repo_analysis['discrepancies'] / repo_analysis['total_files'] * 100).round(2)
    print(repo_analysis)
    
    # File category analysis
    print("\n=== File Category Analysis ===")
    category_analysis = df.groupby('file_category').agg({
        'has_discrepancy': ['count', 'sum']
    })
    
    category_analysis.columns = ['total_files', 'discrepancies']
    category_analysis['discrepancy_rate'] = (category_analysis['discrepancies'] / category_analysis['total_files'] * 100).round(2)
    category_analysis = category_analysis.sort_values('discrepancy_rate', ascending=False)
    print(category_analysis)

In [ ]:
# Create comprehensive visualizations
if not df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Diff Algorithm Comparison Analysis', fontsize=16)
    
    # 1. Overall discrepancy distribution
    discrepancy_counts = df['has_discrepancy'].value_counts()
    axes[0, 0].pie(discrepancy_counts.values, 
                   labels=['No Discrepancy', 'Has Discrepancy'], 
                   autopct='%1.1f%%',
                   colors=['lightgreen', 'lightcoral'])
    axes[0, 0].set_title('Overall Discrepancy Distribution')
    
    # 2. Discrepancies by repository
    repo_discrepancies = df[df['has_discrepancy'] == True]['repository'].value_counts()
    if not repo_discrepancies.empty:
        axes[0, 1].bar(repo_discrepancies.index, repo_discrepancies.values, color='skyblue')
        axes[0, 1].set_title('Discrepancies by Repository')
        axes[0, 1].set_xlabel('Repository')
        axes[0, 1].set_ylabel('Number of Discrepancies')
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Discrepancies by file category
    category_discrepancies = df[df['has_discrepancy'] == True]['file_category'].value_counts()
    if not category_discrepancies.empty:
        axes[1, 0].bar(category_discrepancies.index, category_discrepancies.values, color='lightcoral')
        axes[1, 0].set_title('Discrepancies by File Category')
        axes[1, 0].set_xlabel('File Category')
        axes[1, 0].set_ylabel('Number of Discrepancies')
        axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. Diff size comparison
    if df['histogram_diff_size'].sum() > 0 and df['myers_diff_size'].sum() > 0:
        axes[1, 1].scatter(df['myers_diff_size'], df['histogram_diff_size'], alpha=0.6)
        axes[1, 1].plot([0, df['myers_diff_size'].max()], [0, df['myers_diff_size'].max()], 'r--', alpha=0.7)
        axes[1, 1].set_xlabel('Myers Diff Size (characters)')
        axes[1, 1].set_ylabel('Histogram Diff Size (characters)')
        axes[1, 1].set_title('Diff Size Comparison')
        axes[1, 1].set_xlim(0, None)
        axes[1, 1].set_ylim(0, None)
    
    plt.tight_layout()
    plt.savefig('diff_algorithm_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nVisualization saved as 'diff_algorithm_analysis.png'")
else:
    print("No data available for visualization")

## Detailed Discrepancy Analysis

Looking at specific examples of discrepancies between algorithms.

In [ ]:
if not df.empty:
    # Find examples of discrepancies
    discrepancy_examples = df[df['has_discrepancy'] == True].head(5)
    
    if not discrepancy_examples.empty:
        print("=== Sample Discrepancy Examples ===")
        
        for idx, row in discrepancy_examples.iterrows():
            print(f"\n--- Example {idx + 1} ---")
            print(f"Repository: {row['repository']}")
            print(f"File: {row['file_path']}")
            print(f"Category: {row['file_category']}")
            print(f"Commit: {row['commit_hash'][:8]}")
            print(f"Myers diff size: {row['myers_diff_size']} chars")
            print(f"Histogram diff size: {row['histogram_diff_size']} chars")
            
            # Show first few lines of each diff if they're not too long
            if len(row['myers_diff']) < 500:
                print(f"\nMyers diff excerpt:\n{row['myers_diff'][:200]}...")
            if len(row['histogram_diff']) < 500:
                print(f"\nHistogram diff excerpt:\n{row['histogram_diff'][:200]}...")
    
    # Statistical analysis
    print("\n=== Statistical Analysis ===")
    
    discrepancy_df = df[df['has_discrepancy'] == True]
    
    if not discrepancy_df.empty:
        size_diff = discrepancy_df['histogram_diff_size'] - discrepancy_df['myers_diff_size']
        
        print(f"Average size difference: {size_diff.mean():.2f} characters")
        print(f"Median size difference: {size_diff.median():.2f} characters")
        print(f"Standard deviation: {size_diff.std():.2f} characters")
        
        print(f"\nFiles where Histogram > Myers: {(size_diff > 0).sum()}")
        print(f"Files where Myers > Histogram: {(size_diff < 0).sum()}")
        print(f"Files with same size: {(size_diff == 0).sum()}")
    
    # Save detailed analysis
    analysis_summary = {
        'total_files': len(df),
        'total_discrepancies': df['has_discrepancy'].sum(),
        'discrepancy_rate': (df['has_discrepancy'].sum() / len(df)) * 100,
        'repositories_analyzed': df['repository'].nunique(),
        'file_categories': df['file_category'].nunique()
    }
    
    summary_df = pd.DataFrame([analysis_summary])
    summary_df.to_csv("diff_analysis_summary.csv", index=False)
    
    print(f"\nAnalysis summary saved to 'diff_analysis_summary.csv'")
    print(f"Detailed results saved to 'diff_algorithms_comparison.csv'")
else:
    print("No data available for detailed analysis")

## Results Summary

This notebook provides a comprehensive comparison of diff algorithms (Myers vs Histogram) across multiple open-source repositories.

### Key Findings:

1. **Algorithm Differences**: 
   - Myers algorithm (default git diff)
   - Histogram algorithm (alternative approach)
   - Discrepancies found in diff outputs

2. **File Type Impact**:
   - Source code files vs configuration files
   - Documentation vs resource files
   - Different categories show varying discrepancy rates

3. **Repository Analysis**:
   - Multi-repository comparison
   - Statistical analysis of differences
   - Visualization of patterns

### Key Datasets Generated:
- `retrofit_diff_analysis.csv`: Retrofit repository analysis
- `glide_diff_analysis.csv`: Glide repository analysis  
- `timber_diff_analysis.csv`: Timber repository analysis
- `diff_algorithms_comparison.csv`: Combined analysis results
- `diff_analysis_summary.csv`: Statistical summary

### Applications:
- Understanding diff algorithm behavior
- Tool selection for code analysis
- Quality assessment of diff outputs
- Research in software evolution
- Development of better diff algorithms

### Visualization Output:
- `diff_algorithm_analysis.png`: Comprehensive analysis charts

This analysis helps understand when and why different diff algorithms produce varying outputs, which is crucial for tools that depend on diff analysis for software engineering tasks.